# Xception vanilla

# Mount and extract dataset from google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip" -d "/content/data"

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip"

In [ ]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Loading Data generator function

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Zoom transformations
    horizontal_flip=True  # Horizontal flips
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for testing data

# Define paths to your train, test, and validation directories
train_dir = 'train/'
validation_dir = 'validation/'
test_dir = 'test/'

# Set up data generators to read images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Number of images in each batch
    class_mode='categorical'  # Since you have multiple classes
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Load pre-trained Xception model (excluding top layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

NUM_CLASSES = 38

# Add custom top layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer with 1024 units
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # Output layer with softmax activation

# Combine base model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
# model.summary()

# Check GPU availability

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Check TensorFlow GPU support
print("TensorFlow GPU Support:", tf.test.is_built_with_cuda())

# Train

In [ ]:
# Define the number of training and validation steps per epoch
train_steps_per_epoch = len(train_generator)
validation_steps_per_epoch = len(validation_generator)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch
)

# Evaluate the model on the test set
test_steps_per_epoch = len(test_generator)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Evaluation

## Evaluation Metric Calculation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

# Generate predictions for the test set
y_pred_prob = model.predict(test_generator, steps=test_steps_per_epoch)
y_pred = np.argmax(y_pred_prob, axis=1)

# Extract true labels from the test generator
y_true = test_generator.classes


# Print test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

## Accuracy, Precision, Recall, and F1-Score Calculation

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


## ROC-AUC Calculation and Plotting (for Binary Classification)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Calculate ROC-AUC (for binary classification tasks)
if 2 == 2:
    roc_auc = roc_auc_score(y_true, y_pred_prob[:, 1])
    print("ROC-AUC:", roc_auc)

    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob[:, 1])
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()
